# User Study 02 - RL Audio Notebook

Please click the folliwng two links to read the explanatory statrement and answer the pre-study questionnaire.

<span style="color:yellow">**Explanatory Statement:**</span> https://drive.google.com/file/d/1-8npbW1wg_ABzBnnGa1dgEgCaYjDED8o/view?usp=sharing

<span style="color:yellow">**Consent Form:**</span> https://forms.gle/eZMYzwaHG4Bqpgp6A

<span style="color:yellow">**Pre-study Questionnaire:**</span> https://forms.gle/GAU8xzekWKkTMDLVA   (Participant ID Required)

# Setup

<span style="color:red">**Before starting this Notebook...**</span>

1. <span style="color:red">**Sub the line of code specifying PWD path**</span> on your device to: ./../RL_audio/notebooks

2. <span style="color:red">**Install the required packages**</span> (below) to run the notebook

3. <span style="color:red">**Enable JupyterLab Dark**</span>. Under "Settings" --> Theme --> "JupyterLab Dark" (*Optional but recommended*)

In [ ]:
%cd /home/liamroy/Documents/PHD/repos/RL_audio/notebooks

# %cd '/Users/liamroy/Library/CloudStorage/GoogleDrive-liam.roy@monash.edu/My Drive/Robotics/User Study 02 - Auto State Expression/RL_audio/notebooks'

# %cd <add your path here and comment out the others>

In [ ]:
PWD = %pwd

## Packages

Please install the following:

pygame   (see this webpage ~ https://www.pygame.org/wiki/GettingStarted)  
jupyterlab, numpy, termcolor, openpyxl, nbconvert-webpdf  


Either use:    
--> sudo apt-get install <package_name>  
--> python3 -m pip install <package_name>  
--> conda install -c conda-forge <package_name>  


Example using conda:  
--> conda install -c conda-forge <package_name>  

jupyterlab or notebook  
numpy  
termcolor  
openpyxl  
nbconvert-webpdf              

## Imports

In [ ]:
#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# IMPORTS
import os
import shutil
import time
import numpy as np
import random
import argparse
import linecache

from scripts import audio_control_V2 as audio_ctrl
from scripts import ucb1_algorithm_V2 as ucb1
from scripts import misc_helpers_V2 as mischelp

import sys

from termcolor import colored, cprint
# Termcolor guide: https://pypi.org/project/termcolor/


#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#  ARGUMENTS & PARSER (Save this code for scripts working with CLI)

# argParser = argparse.ArgumentParser()

# # Enter any valid integer value
# argParser.add_argument("-b", "--budg", required=False, help="select the budget value (dtype=int)")

# # Enter a valid parameter discritization integer (must match sound library size)
# argParser.add_argument("-d", "--disc", required=False, help="select discritization size (dtype=int)")

# # Enter true if you would like to see hidden print log, including Q-tables
# argParser.add_argument("-p", "--prnt", required=False, help="show hidden print log (dtype=bool)")

# # To load and save, simply enter in the base filename such as "lastsave" or "set_A", system takes care of rest
# argParser.add_argument("-s", "--save", required=False, help="filename to save Q-table on exit (dtype=str)") 
# argParser.add_argument("-l", "--load", required=False, help="load Q-table from filename (dtype=str)") 	

## Initializations

In [ ]:
# Parameter discritization
param_disc = 3 

state_descriptions = ["Stuck	  \t- robot is in trouble and needs your help",         #Index [0]
                      "Accomplished \t- robot has successfully completed it's task",    #Index [1]
                      "Progressing \t- robot is working and doesn't need help",         #Index [2]
                      "None of the above"]                                              #Index [3]


num_of_states = len(state_descriptions) - 1 # Adding a minus 1 since the last state in "state_descriptions" is "none of the above"
state_range = np.arange(num_of_states)


# CREATE SOUND LIBRARY A
# For library A, setup the array using libA
library_A = "libA"

# Create an array of size (N x N x N) where N = number of discretized regions
# number of discretized regions for each param --> i.e. if equals 3 then (0, 1, 2)
# ** must align with the discretization for selected sound library
sound_obj_array_A = np.ndarray((param_disc, param_disc, param_disc),dtype=object)

for param_1_range in range(param_disc):
	for param_2_range in range(param_disc):
		for param_3_range in range(param_disc):
			sound_obj_array_A[param_1_range, param_2_range, param_3_range] = audio_ctrl.audio_object(param_1=param_1_range, param_2=param_2_range, param_3=param_3_range, sound_library=library_A)
			
			
# CREATE SOUND LIBRARY B
# For library B, setup the array using libB
library_B = "libB"

# Create an array of size (N x N x N) where N = number of discretized regions
# number of discretized regions for each param --> i.e. if equals 3 then (0, 1, 2)
# ** must align with the discretization for selected sound library
sound_obj_array_B = np.ndarray((param_disc, param_disc, param_disc),dtype=object)

for param_1_range in range(param_disc):
	for param_2_range in range(param_disc):
		for param_3_range in range(param_disc):
			sound_obj_array_B[param_1_range, param_2_range, param_3_range] = audio_ctrl.audio_object(param_1=param_1_range, param_2=param_2_range, param_3=param_3_range, sound_library=library_B)
			

Start by entering your user ID. 
	
<span style="color:yellow">**Click on the first cell below & hit 'shift + enter'...**</span>

In [ ]:



current_user_ID_str = mischelp.get_user_ID(parent_dir=PWD, num_of_states=num_of_states)




# MAIN STUDY <span style="color:yellow">(start here)</span>

Welcome to this study's <span style="color:yellow">**Jupyter notebook**</span>. In this work, we are developing strategies for improving human-robot interaction with nonverbal sounds (<span style="color:yellow">**_beeps & boops_**</span>).

This study is best completed with <span style="color:yellow">**headphones**</span>. Ensure your volume is on.

While a robot is working on a task, it can have many different internal states. Consider a scenario in which you are guiding a robot as it navigates a maze to find fruit:  

If the robot gets lost or stuck behind an obstacle, the robot's internal state is: <span style="color:Red">**Stuck**</span>

Similarly, if the robot was able to reach it's goal (find a fruit), the robot's internal state is: <span style="color:green">**Accomplished**</span>

If the robot is actively working on the task (navigating through the maze) but has neither gotten stuck nor completed the task, the robot's internal state is: <span style="color:blue">**Progressing**</span>

In this notebook, you will be asked to run through <span style="color:yellow">**3 sections**</span>. In each of these sections, a virtual robot will play a sound. Once you listen to the sound, you will be asked to select which robot state you think the virtual robot is in. You will have the options: <span style="color:Red">**Stuck**</span>, <span style="color:green">**Accomplished**</span>, <span style="color:blue">**Progressing**</span> and <span style="color:orange">**None of the above**</span>.

In addition to each answer, you will also self-score how confident you are in your response, on a scale from 1 to 10. 

This process will repeat several times as a learning algorithm is processing in the background. <span style="color:yellow">**If you have any questions, ask your study moderator**</span>. Have fun!

## SECTION 1A

<span style="color:yellow">**Our first robot is named Jackal.**</span>

Let's listen to <span style="color:yellow">**Jackal**</span> make a few sounds to express itself. 

For each sound, you will asked to select which robot state you think the robot is in.

<span style="color:yellow">**Click on the cell below & hit 'shift + enter'...**</span>

In [ ]:



mischelp.get_user_accuracy(sound_obj_array=sound_obj_array_A, lib_str=library_A, sect_str="sect1", user_ID_str=current_user_ID_str, num_of_states=num_of_states, 
                           states_array=np.ndarray(num_of_states, dtype=object), state_descriptions=state_descriptions, param_disc=param_disc, load_file="pilotset", seed=70,  mixer_volume=0.75)




<img src="images/jackal.png" alt="Jackal Robot" style="height: 334px; width:600px;"/>

## SECTION 1B

<span style="color:yellow">**Our next robot is named the Spot.**</span>

Let's listen to <span style="color:yellow">**Spot**</span> make a few sounds to express itself. 

You will notice <span style="color:yellow">**Spot**</span> sounds slightly different to <span style="color:yellow">**Jackal**</span>. For each sound, you will asked to select which robot state you think the robot is in.

<span style="color:yellow">**Click on the cell below & hit 'shift + enter'...**</span>

In [ ]:


mischelp.get_user_accuracy(sound_obj_array=sound_obj_array_B, lib_str=library_B, sect_str="sect1", user_ID_str=current_user_ID_str, num_of_states=num_of_states, 
                           states_array=np.ndarray(num_of_states, dtype=object), state_descriptions=state_descriptions, param_disc=param_disc, load_file="pilotset", seed=51,  mixer_volume=0.45)




<img src="images/spot.png" alt="Spot Robot" style="height: 334px; width:600px;"/>

## Section 2

In section 2, we'll be listening to <span style="color:yellow">**Jackal**</span> again.

Similar to before, <span style="color:yellow">**Jackal**</span> make a few sounds to express itself, and you will asked to select which robot state you think the robot is in.

This process will repeat several times as a learning algorithm is processing in the background.

### Section 2O

<span style="color:yellow">**Click on the cell below & hit 'shift + enter'...**</span>

In [ ]:
# SECTION 2O


ucb1.ucb1_algor(num_of_states=num_of_states, state_descriptions=state_descriptions, param_disc=param_disc, 
                sound_obj_array=sound_obj_array_A, current_user_ID_str=current_user_ID_str, sect_str="_sect2O", load_file="pilotset", budget=60, 
                delta_Q_thresh=2.0, conv_thresh=3, printer=None, mixer_volume=0.75)


sect3_load_str = current_user_ID_str + "_sect2O_final"

<img src="images/jackal.png" alt="Jackal Robot" style="height: 334px; width:600px;"/>

### Section 2X

<span style="color:yellow">**Click on the cell below & hit 'shift + enter'...**</span>

In [ ]:
# SECTION 2X


ucb1.ucb1_algor(num_of_states=num_of_states, state_descriptions=state_descriptions, param_disc=param_disc, 
                sound_obj_array=sound_obj_array_A, current_user_ID_str=current_user_ID_str, sect_str="_sect2X", load_file=None, budget=60, 
                delta_Q_thresh=2.0, conv_thresh=3, printer=None, mixer_volume=0.75)


sect3_load_str = current_user_ID_str + "_sect2X_final"

<img src="images/jackal.png" alt="Jackal Robot" style="height: 334px; width:600px;"/>

## Section 3A

We're nearly finished ~ <span style="color:yellow">**home stretch!**</span>

Let's listen to <span style="color:yellow">**Jackal**</span> express itself one last time. 

For each sound, you will asked to select which robot state you think the robot is in.

<span style="color:yellow">**Click on the cell below & hit 'shift + enter'...**</span>

In [ ]:


mischelp.get_user_accuracy(sound_obj_array=sound_obj_array_A, lib_str=library_A, sect_str="sect3", user_ID_str=current_user_ID_str, num_of_states=num_of_states, 
                           states_array=np.ndarray(num_of_states, dtype=object), state_descriptions=state_descriptions, param_disc=param_disc, load_file=sect3_load_str, seed=51, mixer_volume=0.75)




<img src="images/jackal.png" alt="Jackal Robot" style="height: 334px; width:600px;"/>

## Section 3B

Lastly, let's listen to <span style="color:yellow">**Spot**</span> express itself one last time.  

You will notice <span style="color:yellow">**Spot**</span> sounds slightly different to <span style="color:yellow">**Jackal**</span>. For each sound, you will asked to select which robot state you think the robot is in.

<span style="color:yellow">**Click on the cell below & hit 'shift + enter'...**</span>

In [ ]:



mischelp.get_user_accuracy(sound_obj_array=sound_obj_array_B, lib_str=library_B, sect_str="sect3", user_ID_str=current_user_ID_str, num_of_states=num_of_states, 
                           states_array=np.ndarray(num_of_states, dtype=object), state_descriptions=state_descriptions, param_disc=param_disc, load_file=sect3_load_str, seed=48, mixer_volume=0.45)




<img src="images/spot.png" alt="Spot Robot" style="height: 334px; width:600px;"/>

## Final Results: <span style="color:yellow">**Jackal**</span>

Lets listen to the sounds the Jackal learned to express each of it's 3 states...

<span style="color:yellow">**Click on the cell below & hit 'shift + enter'...**</span>

In [ ]:
# State 0 - Stuck


mischelp.play_final_sound(sound_obj_array=sound_obj_array_A, lib_str=library_A, sect_str="sect3", user_ID_str=current_user_ID_str, given_state_idx=0, 
                           states_array=np.ndarray(num_of_states, dtype=object), state_descriptions=state_descriptions, param_disc=param_disc, load_file=sect3_load_str, seed=48, mixer_volume=0.45)




In [ ]:
# State 1 - Accomplished


mischelp.play_final_sound(sound_obj_array=sound_obj_array_A, lib_str=library_A, sect_str="sect3", user_ID_str=current_user_ID_str, given_state_idx=1, 
                           states_array=np.ndarray(num_of_states, dtype=object), state_descriptions=state_descriptions, param_disc=param_disc, load_file=sect3_load_str, seed=48, mixer_volume=0.45)




In [ ]:
# State 2 - Progressing


mischelp.play_final_sound(sound_obj_array=sound_obj_array_A, lib_str=library_A, sect_str="sect3", user_ID_str=current_user_ID_str, given_state_idx=2, 
                           states_array=np.ndarray(num_of_states, dtype=object), state_descriptions=state_descriptions, param_disc=param_disc, load_file=sect3_load_str, seed=48, mixer_volume=0.45)




## Final Results: <span style="color:yellow">**Spot**</span>

Lets listen to the sounds the Jackal learned to express each of it's 3 states...

<span style="color:yellow">**Click on the cell below & hit 'shift + enter'...**</span>

In [ ]:
# State 0 - Stuck


mischelp.play_final_sound(sound_obj_array=sound_obj_array_B, lib_str=library_B, sect_str="sect3", user_ID_str=current_user_ID_str, given_state_idx=0, 
                           states_array=np.ndarray(num_of_states, dtype=object), state_descriptions=state_descriptions, param_disc=param_disc, load_file=sect3_load_str, seed=48, mixer_volume=0.45)




In [ ]:
# State 1 - Accomplished


mischelp.play_final_sound(sound_obj_array=sound_obj_array_B, lib_str=library_B, sect_str="sect3", user_ID_str=current_user_ID_str, given_state_idx=1, 
                           states_array=np.ndarray(num_of_states, dtype=object), state_descriptions=state_descriptions, param_disc=param_disc, load_file=sect3_load_str, seed=48, mixer_volume=0.45)




In [ ]:
# State 2 - Progressing


mischelp.play_final_sound(sound_obj_array=sound_obj_array_B, lib_str=library_B, sect_str="sect3", user_ID_str=current_user_ID_str, given_state_idx=2, 
                           states_array=np.ndarray(num_of_states, dtype=object), state_descriptions=state_descriptions, param_disc=param_disc, load_file=sect3_load_str, seed=48, mixer_volume=0.45)




# Save the Output 

Run the following code block to save the output of this Jupyter Notebook.

<span style="color:yellow">**Click on the cell below & hit 'shift + enter'...**</span>

In [ ]:

file_path_name = "user_data/user_" + current_user_ID_str + "/final_output"

cmd = "jupyter nbconvert --to webpdf --allow-chromium-download study_notebook_V4.ipynb --output " + file_path_name
if(os.system(cmd)):
	print("Error converting to .py")
	print(f"cmd: {cmd}")
    

# Closing Survey

Please click the folliwng link to answer a short post-study questionnaire.

<span style="color:yellow">**Pre-study Questionnaire:**</span> https://forms.gle/K6RnncY82vSVdyE38   (Participant ID Required)

Thank you for completing this Jupyter Notebook. 

### NOTES & DEBUG

<span style="color:red">**This section is not part of the survey.**</span>

In [ ]:
# PILOTSET ARRAY VALUE SETTER

# State 0: Stuck - Pilot Set
manual_Qtable_state_0 = np.array([[[1., -1., -3.], [2., 0., -3.], [3., 2., -3.]], 
								  [[2., -1., -3.], [2., 0., -3.], [4., 2., -3.]],
								  [[2., -1., -3.], [3., 0., -3.], [5., 3., -3.]]]) * 1.0

print("State 0: Stuck")
print(manual_Qtable_state_0.shape, "\n")
print(manual_Qtable_state_0, "\n")



# State 1: Successful - Pilot Set
manual_Qtable_state_1 = np.array([[[-3., 0., 2.], [-3., 1., 3.], [-3., 0., 2.]], 
								  [[-3., 0., 4.], [-3., 1., 5.], [-3., 0., 3.]],
								  [[-3., 0., 2.], [-3., 1., 3.], [-3., 0., 2.]]]) * 1.0

print("State 1: Successful")
print(manual_Qtable_state_1.shape, "\n")
print(manual_Qtable_state_1, "\n")



# State 2: Progressing - Pilot Set
manual_Qtable_state_2 = np.array([[[0., 3., 0.], [-3., 2., -3.], [-3., 1., -3.]], 
								  [[0., 5., 0.], [-3., 3., -3.], [-3., 1., -3.]],
								  [[0., 4., 0.], [-3., 2., -3.], [-3., 1., -3.]]]) * 1.0

print("State 2: Successful")
print(manual_Qtable_state_2.shape, "\n")
print(manual_Qtable_state_2, "\n")



np.save("arrays/pilotset_st0.npy", manual_Qtable_state_0)
np.save("arrays/pilotset_st1.npy", manual_Qtable_state_1)
np.save("arrays/pilotset_st2.npy", manual_Qtable_state_2)

Creating buttons and widgets: https://medium.com/@technologger/how-to-interact-with-jupyter-33a98686f24e

In [ ]:
%whos